# Вежба 7: Mean Shift кластерирање

За да го примениме Mean Shift во scikit-learn го користиме Титаник податочното множество кое базирано на информација за одредени патници на Титаник, како класа, име, пол, возраст, итн. Овие информации ќе ги користиме за да предвидиме дали патникот го преживеал потонувањето на Титаник.
Притоа, ќе се обидеме да најдеме групи со патници со слични карактеристики

In [25]:
import pandas as pd

In [26]:
titanic_data = pd.read_csv('../Exercises 5 - Datasets/titanic.csv', quotechar='"')
titanic_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Preprocessing

Некои од карактеристиките не ни се од корист, па затоа ќе ги исфриле соодветните колони од податочната рамка. Колоната Survived ни кажува дали патникот преживеал (вредност 1) или не (вредност 0).
Колоната PClass ни кажува во која класа патувал (прва, втора или трета)
Age е возраста
Fare колку платил за билетот
SibSp ни кажува колку браќа/сестри патувале со патникот
Parch ни кажува колку родители патувале со патникот
Embarked ни ја кажува првата буква на пристаништето на кое се качил патникот.

In [27]:
titanic_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], 'columns', inplace=True)
titanic_data.head()

C:\Users\dimit\AppData\Local\Temp\ipykernel_6908\901095614.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  titanic_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], 'columns', inplace=True)


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


Има потреба колоната за полото (Sex) да ја трансформира во нумеричка вредност (1-за машки, 0-за женски)

In [28]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
titanic_data['Sex'] = le.fit_transform(titanic_data['Sex'].astype(str))
titanic_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,S
1,1,1,0,38.0,1,0,71.2833,C
2,1,3,0,26.0,0,0,7.9250,S
3,1,1,0,35.0,1,0,53.1000,S
4,0,3,1,35.0,0,0,8.0500,S


Користиме One-hot репрезентација за колоната Embarked 

In [29]:
titanic_data = pd.get_dummies(titanic_data, columns=['Embarked'])
titanic_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,22.0,1,0,7.2500,0,0,1
1,1,1,0,38.0,1,0,71.2833,1,0,0
2,1,3,0,26.0,0,0,7.9250,0,0,1
3,1,1,0,35.0,1,0,53.1000,0,0,1
4,0,3,1,35.0,0,0,8.0500,0,0,1


Правиме проверка дали имаме Null податоци. Имаме 177 редови каде недостасуваат одредени податоци

In [30]:
titanic_data[titanic_data.isnull().any(axis=1)]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
5,0,3,1,NaN,0,0,8.4583,0,1,0
17,1,2,1,NaN,0,0,13.0000,0,0,1
19,1,3,0,NaN,0,0,7.2250,1,0,0
26,0,3,1,NaN,0,0,7.2250,1,0,0
28,1,3,0,NaN,0,0,7.8792,0,1,0
...,...,...,...,...,...,...,...,...,...,...
859,0,3,1,NaN,0,0,7.2292,1,0,0
863,0,3,0,NaN,8,2,69.5500,0,0,1
868,0,3,1,NaN,0,0,9.5000,0,0,1
878,0,3,1,NaN,0,0,7.8958,0,0,1


Ги чистиме тие редови

In [31]:
titanic_data = titanic_data.dropna()

## Mean Shift Clustering

Откако го процесиравме податочното множество, инстанцираме естиметор за Mean Shift алгоритам за кластерирање.
Единствен хипер-параметар e bandwidth кој е правопропорционален со стандардната девијација.
Со методот fit го започнуваме тренирањето

In [32]:
from sklearn.cluster import MeanShift

analyzer = MeanShift(bandwidth=50)
analyzer.fit(titanic_data)

MeanShift(bandwidth=50)

Во претходниот код ставивме случајна вредност за bandwidth параметарот. Постои помошна функција estimate_bandwidth која дава добра вредност за овој хипер-параметар. 30 е вистинската вредност.

In [33]:
from sklearn.cluster import estimate_bandwidth

estimate_bandwidth(titanic_data)

30.44675914497196

Да го видиме бројот на кластери кои беа одредени со алгоритмот за кластерирање

In [34]:
labels = analyzer.labels_

In [35]:
import numpy as np

np.unique(labels)

array([0, 1, 2], dtype=int64)

Да додадеме нова колона cluster_group во која ќе наведеме во кој кластер припаѓа даден патник

In [36]:
titanic_data['cluster_group'] = np.nan
data_length = len(titanic_data)

#pd.set_option('mode.chained_assignment',None)

for i in range(data_length):
    titanic_data.iloc[i, titanic_data.columns.get_loc('cluster_group')] = labels[i]

In [37]:
titanic_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,cluster_group
0,0,3,1,22.0,1,0,7.2500,0,0,1,0.0
1,1,1,0,38.0,1,0,71.2833,1,0,0,0.0
2,1,3,0,26.0,0,0,7.9250,0,0,1,0.0
3,1,1,0,35.0,1,0,53.1000,0,0,1,0.0
4,0,3,1,35.0,0,0,8.0500,0,0,1,0.0


Сега да го видиме целото податочно множество. Од 714 патници, околу 40% преживеале, средната патничка класа е 2.23 (повеќе луѓе биле во класа 2 и 3, во споредба со подобрата класа 1), имало повеќе мажи, средната цена на билетот била околу 35 долари 

In [38]:
titanic_data.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,cluster_group
count,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000
mean,0.406162,2.236695,0.634454,29.699118,0.512605,0.431373,34.694514,0.182073,0.039216,0.775910,0.051821
std,0.491460,0.838250,0.481921,14.526497,0.929783,0.853289,52.918930,0.386175,0.194244,0.417274,0.240040
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,20.125000,0.000000,0.000000,8.050000,0.000000,0.000000,1.000000,0.000000
50%,0.000000,2.000000,1.000000,28.000000,0.000000,0.000000,15.741700,0.000000,0.000000,1.000000,0.000000
75%,1.000000,3.000000,1.000000,38.000000,1.000000,1.000000,33.375000,0.000000,0.000000,1.000000,0.000000
max,1.000000,3.000000,1.000000,80.000000,5.000000,6.000000,512.329200,1.000000,1.000000,1.000000,2.000000


Да ги групираме патниците по кластер и да ја најдеме средната вредност на сите променливи во рамки на еден кластер

In [39]:
titanic_cluster_data = titanic_data.groupby(['cluster_group']).mean()
titanic_cluster_data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
cluster_group,,,,,,,,,,
0.0,0.388235,2.298529,0.651471,29.559191,0.505882,0.404412,25.415625,0.167647,0.041176,0.788235
1.0,0.741935,1.000000,0.258065,32.223226,0.709677,1.032258,192.008732,0.419355,0.000000,0.580645
2.0,1.000000,1.000000,0.666667,35.333333,0.000000,0.333333,512.329200,1.000000,0.000000,0.000000


Да ги видиме и бројот на патници во секој кластер. Гледаме дека имаме 2 кластери кои имаат репрезентативен број на патници. Третиот кластер има само 3 патници и не е репрезентативен. 
Во првиот кластер голем дел од патниците не преживеале и патниците биле од полошите класи (2 и 3 класа), има повеќе мажи од жени и цената на билетот е под просекот.
Вториот кластер се состои од голем дел од патниците кои преживеале и тоа биле патници претежно од прва класа и претежно жени кои го платиле билетот доста над просекот

In [40]:
titanic_cluster_data['Counts'] = pd.Series(titanic_data.groupby(['cluster_group']).size())
titanic_cluster_data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Counts
cluster_group,,,,,,,,,,,
0.0,0.388235,2.298529,0.651471,29.559191,0.505882,0.404412,25.415625,0.167647,0.041176,0.788235,680
1.0,0.741935,1.000000,0.258065,32.223226,0.709677,1.032258,192.008732,0.419355,0.000000,0.580645,31
2.0,1.000000,1.000000,0.666667,35.333333,0.000000,0.333333,512.329200,1.000000,0.000000,0.000000,3


Да ги погледнеме податоците за вториот кластер (кластер на преживеани), можете да ја забележите вредноста на билетот што ја платиле, која за тоа време е прилично висока. 

In [41]:
titanic_data[titanic_data['cluster_group'] == 1].describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,cluster_group
count,31.000000,31.0,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.0,31.000000,31.0
mean,0.741935,1.0,0.258065,32.223226,0.709677,1.032258,192.008732,0.419355,0.0,0.580645,1.0
std,0.444803,0.0,0.444803,15.327994,1.006431,1.016001,50.203716,0.501610,0.0,0.501610,0.0
min,0.000000,1.0,0.000000,0.920000,0.000000,0.000000,133.650000,0.000000,0.0,0.000000,1.0
25%,0.500000,1.0,0.000000,22.000000,0.000000,0.000000,151.550000,0.000000,0.0,0.000000,1.0
50%,1.000000,1.0,0.000000,31.000000,0.000000,1.000000,164.866700,0.000000,0.0,1.000000,1.0
75%,1.000000,1.0,0.500000,41.500000,1.000000,2.000000,237.522900,1.000000,0.0,1.000000,1.0
max,1.000000,1.0,1.000000,64.000000,3.000000,4.000000,263.000000,1.000000,0.0,1.000000,1.0


In [42]:
titanic_data[titanic_data['cluster_group'] == 1]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,cluster_group
27,0,1,1,19.00,3,2,263.0000,0,0,1,1.0
88,1,1,0,23.00,3,2,263.0000,0,0,1,1.0
118,0,1,1,24.00,0,1,247.5208,1,0,0,1.0
195,1,1,0,58.00,0,0,146.5208,1,0,0,1.0
268,1,1,0,58.00,0,1,153.4625,0,0,1,1.0
269,1,1,0,35.00,0,0,135.6333,0,0,1,1.0
297,0,1,0,2.00,1,2,151.5500,0,0,1,1.0
299,1,1,0,50.00,0,1,247.5208,1,0,0,1.0
305,1,1,1,0.92,1,2,151.5500,0,0,1,1.0
311,1,1,0,18.00,2,2,262.3750,1,0,0,1.0


## Самостојна работа

Пробајте го алгоритмот со bandiwdth=30 и анализирајте ги резултатите.
Колку кластери добивате вака?
Колку од овие кластери се репрезентативни?
Пробајте да ги објасните кластерите преку информациите за патниците во соодветните кластери

### Ги предаваме податоците на MeanShift алгоритмот

In [50]:
from sklearn.cluster import MeanShift

analyzer = MeanShift(bandwidth=30)
analyzer.fit(titanic_data)

MeanShift(bandwidth=30)

### Гледаме колку категории добивме

In [47]:
labels = analyzer.labels_
import numpy as np

np.unique(labels)

array([0, 1, 2, 3, 4], dtype=int64)

### Додаваме нова колона во DataFrame-от за групата на кластерот

In [48]:
titanic_data['cluster_group'] = np.nan
data_length = len(titanic_data)
for i in range(data_length):
    titanic_data.iloc[i, titanic_data.columns.get_loc('cluster_group')] = labels[i]

### Гледаме (Counts) колку патници припаѓаат по секој кластер

In [49]:
titanic_cluster_data = titanic_data.groupby(['cluster_group']).mean()
titanic_cluster_data['Counts'] = pd.Series(titanic_data.groupby(['cluster_group']).size())
titanic_cluster_data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Counts
cluster_group,,,,,,,,,,,
0.0,0.336918,2.525090,0.679211,28.256720,0.439068,0.370968,15.434139,0.121864,0.046595,0.831541,558
1.0,0.611111,1.296296,0.527778,36.148148,0.814815,0.500000,65.622688,0.333333,0.018519,0.629630,108
2.0,0.733333,1.000000,0.366667,32.430667,0.600000,0.866667,131.183883,0.500000,0.000000,0.500000,30
3.0,0.733333,1.000000,0.266667,30.333333,1.000000,1.333333,239.991940,0.533333,0.000000,0.466667,15
4.0,1.000000,1.000000,0.666667,35.333333,0.000000,0.333333,512.329200,1.000000,0.000000,0.000000,3
